# Problem

Popular question and answer (qna) site -  stackoverflow (+ their sister sites) allows for download of monthly data dumps from https://archive.org/details/stackexchange.

With this data, can we classify the questions/answers based on

* Conceptual v/s howto question 
* Beginner v/s intermediate v/s hard/trick
* A particular question is associated with another question in terms of the next things to do or perhaps the pre-requisites?
* Predict the next question a user may ask based on this current search

The taxanomy could be a useful layout of the land for a student of the area.

# Schema

The schema for their data is located @ https://ia800500.us.archive.org/22/items/stackexchange/readme.txt.
    
Unfortunately, the data is dumped in an XML format and there is preliminary effort to convert that data into CSV format. We have written a converter (convert2csv.py) for the tables of interest.

The schemas for the tables of interest are shown below.


## Posts
-----------
- Id
- PostTypeId
  - 1: Question
  - 2: Answer
- ParentID (only present if PostTypeId is 2)
- AcceptedAnswerId (only present if PostTypeId is 1)
- CreationDate
- Score
- ViewCount
- Body
- OwnerUserId
- LastEditorUserId
- LastEditorDisplayName="Jeff Atwood"
- LastEditDate="2009-03-05T22:28:34.823"
- LastActivityDate="2009-03-11T12:51:01.480"
- CommunityOwnedDate="2009-03-11T12:51:01.480"
- ClosedDate="2009-03-11T12:51:01.480"
- Title=
- Tags=
- AnswerCount
- CommentCount
- FavoriteCount

## Comments
---------------------------
- Id
- PostId
- Score
- Text, e.g.: "@Stu Thompson: Seems possible to me - why not try it?"
- CreationDate, e.g.:"2008-09-06T08:07:10.730"
- UserId

## Post History
---------------------------
- Id
- PostHistoryTypeId
    - 1: Initial Title - The first title a question is asked with.
    - 2: Initial Body - The first raw body text a post is submitted with.
    - 3: Initial Tags - The first tags a question is asked with.
    - 4: Edit Title - A question's title has been changed.
    - 5: Edit Body - A post's body has been changed, the raw text is stored here as markdown.
    - 6: Edit Tags - A question's tags have been changed.
    - 7: Rollback Title - A question's title has reverted to a previous version.
    - 8: Rollback Body - A post's body has reverted to a previous version - the raw text is stored here.
    - 9: Rollback Tags - A question's tags have reverted to a previous version.
    - 10: Post Closed - A post was voted to be closed.
    - 11: Post Reopened - A post was voted to be reopened.
    - 12: Post Deleted - A post was voted to be removed.
    - 13: Post Undeleted - A post was voted to be restored.
    - 14: Post Locked - A post was locked by a moderator.
    - 15: Post Unlocked - A post was unlocked by a moderator.
    - 16: Community Owned - A post has become community owned.
    - 17: Post Migrated - A post was migrated.
    - 18: Question Merged - A question has had another, deleted question merged into itself.
    - 19: Question Protected - A question was protected by a moderator
    - 20: Question Unprotected - A question was unprotected by a moderator
    - 21: Post Disassociated - An admin removes the OwnerUserId from a post.
    - 22: Question Unmerged - A previously merged question has had its answers and votes restored.
- PostId
- RevisionGUID: At times more than one type of history record can be recorded by a single action.  
- CreationDate: "2009-03-05T22:28:34.823"
- UserId
- UserDisplayName: populated if a user has been removed and no longer referenced by user Id
- Comment: This field will contain the comment made by the user who edited a post
- Text: A raw version of the new value for a given revision. 
- CloseReasonId
    - 1: Exact Duplicate - This question covers exactly the same ground as earlier questions on this topic; its answers may be merged with another identical question.
    - 2: off-topic
    - 3: subjective
    - 4: not a real question
    - 7: too localized
       
       
## Users
---------------------------
 - Id
 - Reputation
 - CreationDate
 - DisplayName
 - EmailHash
 - LastAccessDate
 - WebsiteUrl
 - Location
 - Age
 - AboutMe
 - Views
 - UpVotes
 - DownVotes
       

# Conversion from XML to CSV

Run python convert2csv.py to convert each of the xml files to their CSV equivalents. For columns/attributes which contain textual data, the converter encodes them with base64 encoding so that handling of quotes and special characters (separators) is avoided. 

When the data is read back into the dataframe, the corresponding decode (from base64) needs to happen. The converter also creates a sample file of 100 rows for each xml data dump converted.

In [14]:
#imports
import pandas as pd
import base64
import math
import re
import gensim
from gensim import corpora, models
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup 
from nltk.stem.porter import PorterStemmer

%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import pyLDAvis
import pyLDAvis.gensim


SAMPLE_SIZE = 500

In [15]:
posts = pd.read_csv('posts.csv',nrows=SAMPLE_SIZE).dropna(subset=['Body','Title'])
posts['Body'] = posts['Body'].apply(lambda x : BeautifulSoup(base64.b64decode(x),"lxml").get_text())
posts['Title'] = posts['Title'].apply(lambda x : BeautifulSoup(base64.b64decode(x),"lxml").get_text())
posts[['Body','Title', 'Tags']].head(5)

,Body,Title,Tags
0,When should I use can? When should I use could...,"When do I use ""can"" or ""could""?",<word-choice><tenses><politeness><subjunctive-...
1,"Doesn't ""quint"" mean ""five""? What does that h...","Where does the ""quint"" in ""quintessential"" com...",<etymology>
2,"Which is the correct use of these two words, a...","When should I use ""shall"" versus ""will""?",<future-tense><shall-future><will-future>
4,"I think most folk happily use either ""while"" o...","When did ""while"" and ""whilst"" become interchan...",<word-choice><grammar><etymology><conjunctions>
5,\n\nI may not be coming in tomorrow... \nI mig...,"""May"" & ""Might"": What's the right context?",<word-choice><grammar><modal-verbs><auxiliary-...


### Notice that we have stripped out the html formatting tags with BeautifulSoup before reassigning back to the dataframe

In [16]:
comments = pd.read_csv('comments.csv',nrows=SAMPLE_SIZE).dropna()
comments['Text'] = comments['Text'].apply(lambda x : BeautifulSoup(base64.b64decode(x),"lxml").get_text())
comments[['Score','Text']].head(5)

,Score,Text
0,9,I think you need to edit the title of your que...
1,12,It's correct when you're accessing a method of...
2,2,"Yes, I would think in almost any context where..."
3,0,"Would you say `It can certainly be ""acceptable..."
4,4,@serg555: Would you expect anything less on a ...


In [17]:
#posthistory = pd.read_csv('posthistory.csv',nrows=SAMPLE_SIZE).dropna(subset=['Text'])
#posthistory['Text'] = posthistory['Text'].apply(lambda x : BeautifulSoup(base64.b64decode(x),"lxml").get_text())
#posthistory[['Text']].head(5)

In [28]:
users = pd.read_csv('users.csv',nrows=SAMPLE_SIZE).dropna(subset=['AboutMe','Location'])
users['AboutMe'] = users['AboutMe'].apply(lambda x : BeautifulSoup(base64.b64decode(x),"lxml").get_text())
users['Location'] = users['Location'].apply(lambda x : BeautifulSoup(base64.b64decode(x),"lxml").get_text())
users[['Location','AboutMe']].head(5)

,Location,AboutMe
0,on the server farm,"Hi, I'm not really a person.\nI'm a background..."
1,"Corvallis, OR",Developer on the Stack Overflow team. Find me...
2,"New York, NY",Developer on the Stack Overflow team.\nWas dub...
3,"Raleigh, NC",I design stuff for Stack Exchange. Also a prof...
4,California,"I slip my front end into the back end, and the..."


## Further cleansing

* Remove (html) tags & carriage returns from the Text field
* Remove stop words (pick up the nltk stop words)
* Use PorterStemmer to stem words

In [27]:
#global
p_stemmer = PorterStemmer()
stop_words = stopwords.words('english')
stop_words.append('use')
#print(stop_words)

In [20]:
class SentenceTokens():
    def __init__(self,df,field):
        self.field = field
        self.df = df
    
    def __iter__(self):
      for index, row in self.df.iterrows():
         raw_sentence = row[self.field].replace('\n','').lower()
         raw_tokens = filter(None, re.split("[ ?.,\"\']+",raw_sentence))
         #stem_tokens = [p_stemmer.stem(tok) for tok in raw_tokens]
         yield [tok for tok in raw_tokens if not tok in stop_words]

#all posts is a list of (list of tokens). The inner list of tokens is created once for each post
allposts = SentenceTokens(posts,'Body')

#for p in allposts:
#    print(p)

In [21]:
#How frequently each term occurs within each document? We construct a document-term matrix.
dictionary = corpora.Dictionary(allposts)
#print(dictionary.token2id) #maps ids to tokens

In [22]:
#bag of words
corpus = [dictionary.doc2bow(text) for text in allposts]
#corpus, is a list of vectors equal to the number of documents. 
#In each document vector is a series of tuples. 
print(corpus[0])
#(term ID, term frequency) pairs

[(0, 1), (1, 1), (2, 1)]


In [23]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=30)

#num_topics: required. An LDA model requires the user to determine how many topics should be generated. 
#id2word: required. The LdaModel class requires our previous dictionary to map ids to strings.
#passes: optional. The number of laps the model will take through corpus.

In [24]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

In [29]:
@interact(num_topics=5, num_words=6)
def understand(num_topics, num_words):
    return ldamodel.print_topics(num_topics, num_words)
    
#Each generated topic is separated by a comma.
#Within each topic are the three most probable words to appear in that topic.

#interact(understand, x=widgets.IntSlider(min=0,max=300,step=1,value=10));

[(5,
  u'0.017*people + 0.015*correct + 0.013*used + 0.013*different + 0.011*say + 0.010*english'),
 (9,
  u'0.030*english + 0.010*one + 0.010*language + 0.009*like + 0.009*people + 0.008*words'),
 (3,
  u'0.016*try + 0.011*seems + 0.011*whilst + 0.011*pretty + 0.011*often + 0.011*\u201cemployee\u201d'),
 (0,
  u'0.014*one + 0.014*english + 0.013*= + 0.011*written + 0.009*correct + 0.009*people'),
 (8,
  u'0.011*instead + 0.011*right + 0.011*may + 0.008*2010april + 0.008*word + 0.008*sentences')]

In [26]:
sovizdata = pyLDAvis.gensim.prepare(ldamodel,corpus,dictionary)
pyLDAvis.display(sovizdata)


## References

* https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html
* This dashboard @ https://github.com/dhruvaray/soml
    